In [59]:
import findspark
findspark.init('/usr/local/spark/spark-2.3.2-bin-hadoop2.7')

import pyspark
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import pyspark.sql.functions as f
from pyspark.sql.types import DateType, StringType, IntegerType
from pyspark.sql import *
from pyspark.sql.window import Window
import csv
spark = SparkSession \
    .builder \
    .appName("Poolminers") \
    .getOrCreate()

from datetime import datetime
import gc


sc=SparkContext.getOrCreate(spark)


In [4]:
# read warehouse
df = spark.read.parquet('../data/pool_warehouse.parquet')

In [54]:
# FIND TIER 1 ADDRESSES
threshold_tx_paid_out = 10
threshold_blocks_mined_per_day = 2.5 # Percentage
def make_tier1_miner_list(df,threshold_tx_paid_out=10,threshold_blocks_mined_per_day=2.5):
    # find all miners in period and make list
    miner_list = [i.miner_address for i in df.select('miner_address').distinct().collect()]
    # Count transactions paid out per day: group transactions by date and miner
    # tier 1 = percentage mined per day > threshold || transactions paid out > threshold per day# make unique list of tier 1
    df_temp = df.groupby('from_addr','block_timestamp').agg({'to_addr':'count'})
    df_temp = df_temp.dropna()
    # find daily mean
    df_temp = df_temp.groupby('from_addr').agg({'count(to_addr)':'mean'})
    df_temp = df_temp.filter(df_temp['avg(count(to_addr))']>=threshold_tx_paid_out)
    # make list of tier 1 using tx paid out
    list_a = [i.from_addr for i in df_temp.select('from_addr').distinct().collect()]
    # check against miner list to ensure that only miners are included
    list_a = list(set(miner_list) & set(list_a))
    del df_temp
    
    # Get percentage blocks mined per day: group by miner address, day and count
    df_temp = df.groupby('miner_address','block_timestamp')\
        .agg({'block_number':'count'})\
        .withColumn('percent',100*(col('count(block_number)')/
                                   sum(col('count(block_number)')).over(Window.partitionBy())))
    df_temp = df_temp.groupby('miner_address').agg({'percent':'mean'})
    df_temp = df_temp.filter(df_temp['avg(percent)']>=threshold_blocks_mined_per_day)
    list_b = [i.miner_address for i in df_temp.select('miner_address').distinct().collect()]
    print(list_b)
    #merge lists, drop duplicates
    tier1_miner_list = list(set(list_a+list_b))
    del list_a,list_b

    #check this list again miner_address
    gc.collect()
    return tier1_miner_list

tier1_miner_list = make_tier1_miner_list(df)

# save to csv
csvfile = "../data/tier1_miners.csv"
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerow(['miner_address'])
    for val in tier1_miner_list:
        writer.writerow([val])    


['a08091ab0325e384ac45e560d2f85e4b741363aa98881d52d54233a02b33fcaa']


In [61]:
# ADD A COLUMN CALLED POOL_TIER: 1 , 2
pool_tier_udf = f.udf(lambda miner_address: 1 if 
                      miner_address in tier1_miner_list else 2, IntegerType())
df = df.withColumn('pool_tier',pool_tier_udf(df.miner_address))

+---------+
|pool_tier|
+---------+
|        2|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        2|
|        1|
|        1|
|        1|
|        1|
+---------+
only showing top 20 rows



In [70]:
# WRITE TO PARQUET
df.write.parquet("../data/data_warehouse.parquet",compression='gzip')

AnalysisException: 'path file:/home/andre/aion/data_science/spark/jasonz/data/pool_warehouse.parquet already exists.;'

In [71]:
print(df.count())

92521
